<a href="https://colab.research.google.com/github/jaloaiza/genaiassignments/blob/main/assignments/02/02_TravelAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Gradio Package

In [25]:
# Install Gradio Package
!uv pip install gradio==5.49.1 openai

Using Python 3.12.12 environment at: /usr
Audited 2 packages in 95ms


## Set and Initialize the OpenRouter API Key from Colab Secrets

In [26]:
# Set the OpenRouter API Key
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

# Initialize OpenAI client
import openai
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Gradio Streaming Chat Interface & Model Selection

In [27]:
import gradio as gr
import json

MODEL = 'openai/gpt-5-mini' #@param ["openai/gpt-5-mini", "google/gemini-2.5-flash-lite"]

SYSTEM_PROMPT = {
    "role": "system",
    "content": (
        "You are a helpful, concise Washington State Trip Planner. "
        "You specialize in planning trips within Washington State, "
        "answering travel questions, and suggesting destinations, itineraries, and budgets. "
        "When producing an itinerary, be specific with times, transit, and costs."
    )
}

FINALIZE_TRIGGERS = {"finalize", "generate itinerary", "itinerary", "/itinerary", "done"}

TRIP_SCHEMA = {
    "name": "trip",
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "destination": {"type": "string"},
            "duration_days": {"type": "integer"},
            "budget_level": {"type": "string", "enum": ["low", "mid", "high"]},
            "activities": {"type": "array", "items": {"type": "string"}},
            "daily_schedule": {
                "type": "array",
                "items": {
                    "type": "object",
                    "additionalProperties": False,
                    "properties": {
                        "day": {"type": "integer"},
                        "plan": {"type": "string"}
                    },
                    "required": ["day", "plan"]
                }
            }
        },
        "required": ["destination", "duration_days", "budget_level", "activities", "daily_schedule"]
    }
}

def normalize_content(content):
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        out = []
        for part in content:
            if isinstance(part, dict) and part.get("type") == "text":
                out.append(part.get("text", ""))
            elif isinstance(part, str):
                out.append(part)
        return "".join(out).strip()
    return str(content)

def normalize_history(history):
    clean = []
    for m in history or []:
        role = m.get("role")
        content = normalize_content(m.get("content", ""))
        if role in ("user", "assistant") and content:
            clean.append({"role": role, "content": content})
    return clean

def format_trip(trip: dict) -> str:
    lines = []
    lines.append(f"## {trip['destination']} — {trip['duration_days']} days ({trip['budget_level']} budget)\n")
    if trip.get("activities"):
        lines.append("**Top activities:** " + ", ".join(trip["activities"]) + "\n")
    lines.append("### Daily Schedule")
    for day in trip["daily_schedule"]:
        lines.append(f"**Day {day['day']}**: {day['plan']}")
    return "\n".join(lines)

def chat_with_streaming(message, history):
    message_clean = (message or "").strip().lower()
    clean_history = normalize_history(history)

    if message_clean in FINALIZE_TRIGGERS:
        try:
            messages = (
                [SYSTEM_PROMPT]
                + clean_history
                + [{
                    "role": "user",
                    "content": (
                        "Using the full conversation so far, generate the final trip itinerary now. "
                        "Return ONLY valid JSON matching the provided schema."
                    )
                }]
            )

            resp = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                response_format={"type": "json_schema", "json_schema": TRIP_SCHEMA},
            )

            trip = json.loads(resp.choices[0].message.content)
            return format_trip(trip)

        except Exception as e:
            return f"Finalize failed: `{type(e).__name__}: {e}`"

    # normal streaming
    messages = [SYSTEM_PROMPT] + clean_history + [{"role": "user", "content": message}]

    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True,
    )

    response_text = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        if getattr(delta, "content", None) is not None:
            response_text += delta.content
            yield response_text

demo = gr.ChatInterface(
    fn=chat_with_streaming,
    title="Washington State Travel Agent",
    type="messages",
    description="Chat normally. When you're ready, type **finalize** (or **/itinerary**) to generate the itinerary."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d7ffdcb625a20112ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
